<a href="https://colab.research.google.com/github/kyochanpy/forest_bears/blob/main/nb_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
!pip install jismesh
!pip install folium

     |████████████████████████████████| 1.0 MB 9.7 MB/s 
     |████████████████████████████████| 6.3 MB 43.0 MB/s 
     |████████████████████████████████| 15.4 MB 43.5 MB/s 


In [ ]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


import geopandas as gpd

import jismesh.utils as ju
import folium
from shapely.geometry import Polygon
import branca.colormap as cm
from sympy.geometry import *

In [ ]:
def get_population_mesh4_features_by_years(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    y = path[-13:-9]
    print(y)
    if y == "1995":
        p = "T000752001"
    elif y == "2000":
        p = "T000386001"
    elif y == "2005":
        p = "T000387001"
    elif y == "2010":
        p = "T000609001"
    elif y == "2015":
        p = "T000847001"
    print(p)

    output_df = df.loc[:, ("KEY_CODE", p)].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={p:"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df

In [ ]:
def get_geometry_features(input_df, path):
    input_df_ = input_df.copy()
    geo_df = gpd.read_file(path)

    geo_df["KEY_CODE"] = geo_df["KEY_CODE"].astype(int)

    output_df = pd.merge(input_df_, geo_df.iloc[:, [0,-1]], on="KEY_CODE")

    return output_df

In [ ]:
def make_df():
    c = 0
    for i in range(1995, 2016, 5):
        if c == 0:
            KC = get_population_mesh4_features_by_years(f"/content/drive/MyDrive/forest_bears/{str(i)}_6441.txt")[["KEY_CODE"]]
            KC[f"dummy_{str(i)}"] = int(0)
            KEY_CODE = KC
            c += 1
        else:
            KC = get_population_mesh4_features_by_years(f"/content/drive/MyDrive/forest_bears/{str(i)}_6441.txt")[["KEY_CODE"]]
            KC[f"dummy_{str(i)}"] = int(0)
            KEY_CODE = pd.merge(KEY_CODE, KC, how="outer")

    output_df = get_geometry_features(KEY_CODE, "/content/drive/MyDrive/forest_bears/MESH06441_4")

    return output_df.fillna(int(1))

In [ ]:
a = make_df()

1995
T000752001
2000
T000386001
2005
T000387001
2010
T000609001
2015
T000847001


In [ ]:
def get_coordinates_center(input_df):
    input_df_ = input_df.copy()

    lng_center_list = []
    lat_center_list = []
    for n in input_df_["geometry"]:
        coordinates_list = list(zip(*n.exterior.coords.xy))[:-1]
        lng_list_ = []
        lat_list_ = []
        for m in coordinates_list:
            lng_list_.append(m[0])
            lat_list_.append(m[1])
        lng_mean = sum(lng_list_) / (len(lng_list_))
        lng_center_list.append(lng_mean)
        lat_mean = sum(lat_list_) / (len(lat_list_))
        lat_center_list.append(lat_mean)

    output_df = input_df.copy()
    output_df["lng_center"] = lng_center_list
    output_df["lat_center"] = lat_center_list
    
    return output_df

In [ ]:
b = get_coordinates_center(a)

In [ ]:
# 任意の点が15角形で表された札幌市内に含まれているか判別し、内側の点だけのDataFrameを出力する関数
def get_sapporo_df(input_df):
    input_df_ = input_df.copy()

    # 札幌を表すpolygon
    poly = Polygon( *((43.023028, 140.980742),
                      (42.886595, 141.036161),
                      (42.821037, 141.090321),
                      (42.779450, 141.160855),
                      (42.790860, 141.231606),
                      (42.872928, 141.286189),
                      (42.869773, 141.334252),
                      (42.890012, 141.421411),
                      (43.021005, 141.506059),
                      (43.060363, 141.500538),
                      (43.190986, 141.401330),
                      (43.161179, 141.224740),
                      (43.142071, 141.185851),
                      (43.105577, 141.155691),
                      (43.124406, 141.071563)) ) 
    
    points = []
    for lat, lng in zip(
        input_df_["lat_center"].to_numpy(),
        input_df_["lng_center"].to_numpy()
    ):
        points.append((lat, lng))
    
    sapporo_frag = []
    for p in points:
        if poly.encloses_point(p) == True:
            sapporo_frag.append(1)
        else:
            sapporo_frag.append(0)
    
    input_df_["sapporo_frag"] = sapporo_frag
    output_df = input_df_[input_df_["sapporo_frag"] == 1][input_df.columns]

    return output_df

In [ ]:
sapporo_df = get_sapporo_df(b)

In [ ]:
sapporo_df.dtypes

KEY_CODE         int64
dummy_1995     float64
dummy_2000     float64
dummy_2005     float64
dummy_2010     float64
dummy_2015     float64
geometry      geometry
lng_center     float64
lat_center     float64
dtype: object

In [ ]:
sapporo_df.to_csv('sapporo_df.csv', index=False)

In [ ]:
all_df = pd.read_csv('/content/drive/MyDrive/forest_bears/all_df.csv')

In [ ]:
all_df.dtypes

No.         float64
区            object
発生日時         object
出没場所         object
状況           object
lat         float64
lng         float64
KEY_CODE      int64
年度            int64
年             int64
月             int64
日             int64
曜日           object
時間           object
人口            int64
平均気温        float64
天候（昼）_1      object
天候（昼）_2      object
天候（夜）_1      object
天候（夜）_2      object
geometry     object
dtype: object

In [ ]:
len(all_df["KEY_CODE"].unique())

197

In [ ]:
all_df["KEY_CODE"].unique()

array([644132841, 644132822, 644130733, 644132872, 644131123, 644133122,
       644142053, 644142913, 644131524, 644122724, 644132423, 644132844,
       644142134, 644142254, 644122701, 644131171, 644141054, 644142041,
       644131382, 644132162, 644132422, 644132622, 644132943, 644131381,
       644131661, 644131203, 644151384, 644142162, 644131124, 644142202,
       644140864, 644132654, 644141951, 644132204, 644132582, 644122864,
       644132324, 644121512, 644131492, 644142222, 644131593, 644122874,
       644132214, 644132431, 644132414, 644132232, 644132762, 644132512,
       644151584, 644131594, 644132413, 644132502, 644131592, 644131533,
       644132442, 644142163, 644132083, 644132342, 644142174, 644122783,
       644132192, 644132061, 644132064, 644131372, 644131484, 644131491,
       644123901, 644142063, 644132544, 644131692, 644132401, 644122883,
       644132363, 644131612, 644142071, 644132411, 644132542, 644131302,
       644151353, 644142074, 644132243, 644132441, 

In [ ]:
all_df[all_df["KEY_CODE"] == all_df["KEY_CODE"].unique()]

ValueError: ignored

In [ ]:
# KEY_CODE(4次)を含むデータからそのKEY_CODEの人口の特徴量を生成する関数
def get_population_mesh4_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    output_df = df.loc[:, ("KEY_CODE", "T000847001")].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={"T000847001":"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df

In [ ]:
get_population_mesh4_features("/content/drive/MyDrive/forest_bears/2015_6441.txt")

,KEY_CODE,人口
1,644100052,10
2,644100061,152
3,644100063,66
4,644100064,69
5,644100083,4
...,...,...
6876,644177933,7
6877,644177934,3
6878,644177943,13
6879,644177944,3


In [ ]:
def get_unique_mesh_df(input_df, sapporo_df):
    input_df_ = input_df.copy()
    unique_mesh_df = pd.DataFrame()

    unique_mesh_list = input_df_["KEY_CODE"].unique()
    num_list = []
    for i in unique_mesh_list:
        mesh_len = len(input_df_[input_df_["KEY_CODE"] == i])
        num_list.append(mesh_len)
    unique_mesh_df["KEY_CODE"] = unique_mesh_list
    unique_mesh_df["出没回数"] = num_list

    output_df = pd.merge(sapporo_df, unique_mesh_df, on="KEY_CODE", how="outer")
    output_df = output_df[['dummy_1995', 'dummy_2000', 'dummy_2005', 'dummy_2010', 'dummy_2015', '出没回数']].fillna(0)
    #output_df["total"] = output_df["dummy_1995"] + output_df["dummy_2000"] + output_df["dummy_2005"] + output_df["dummy_2010"] + output_df["dummy_2015"]
    # ↑多重共線性が生じるからダメ

    return output_df

In [ ]:
def get_unique_mesh_df(input_df, sapporo_df):
    input_df_ = input_df.copy()
    unique_mesh_df = pd.DataFrame()

    unique_mesh_df["KEY_CODE"] = input_df_["KEY_CODE"].unique()
    unique_mesh_df["出没ダミー"] = 1

    output_df = pd.merge(sapporo_df, unique_mesh_df, on="KEY_CODE", how="outer")
    output_df = output_df[['dummy_1995', 'dummy_2000', 'dummy_2005', 'dummy_2010', 'dummy_2015', '出没ダミー']].fillna(0)
    #output_df["total"] = output_df["dummy_1995"] + output_df["dummy_2000"] + output_df["dummy_2005"] + output_df["dummy_2010"] + output_df["dummy_2015"]
    # ↑多重共線性が生じるからダメ

    return output_df

In [ ]:
def get_unique_mesh_df(input_df, sapporo_df):
    input_df_ = input_df.copy()
    output_df = pd.DataFrame()

    for i in range(2016, 2022):
        year_df = input_df_[input_df_["年度"] == i]
        year_df["出没ダミー"] = 1
        tmp_df = pd.merge(sapporo_df, year_df, on="KEY_CODE", how="outer")
        tmp_df = tmp_df[['dummy_1995', 'dummy_2000', 'dummy_2005', 'dummy_2010', 'dummy_2015', '出没ダミー']].fillna(0)
        output_df = pd.concat([output_df, tmp_df])

    return output_df

In [ ]:
df = get_unique_mesh_df(all_df, sapporo_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [ ]:
df

,dummy_1995,dummy_2000,dummy_2005,dummy_2010,dummy_2015,出没ダミー
0,0.0,1.0,1.0,1.0,1.0,NaN
1,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,1.0,1.0,1.0,1.0,NaN
3,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,1.0,0.0,1.0,1.0,NaN
...,...,...,...,...,...,...
1810,1.0,1.0,1.0,1.0,0.0,NaN
1811,1.0,1.0,1.0,1.0,0.0,NaN
1812,1.0,1.0,1.0,1.0,0.0,NaN
1813,1.0,1.0,1.0,1.0,0.0,NaN


In [ ]:
df

,dummy_1995,dummy_2000,dummy_2005,dummy_2010,dummy_2015,出没回数
0,0.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...
1867,0.0,0.0,0.0,0.0,0.0,1.0
1868,0.0,0.0,0.0,0.0,0.0,1.0
1869,0.0,0.0,0.0,0.0,0.0,1.0
1870,0.0,0.0,0.0,0.0,0.0,2.0


In [ ]:
statsmodels.api

In [ ]:
import statsmodels as sm

df_X = df.drop('出没ダミー', axis=1)
df_y = df['出没ダミー']

df_X = sm.add_constant(df_X)

model = sm.OLS(df_y, df_X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  出没ダミー   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.466
Date:                Mon, 10 Jan 2022   Prob (F-statistic):           5.21e-06
Time:                        08:15:28   Log-Likelihood:                 1174.0
No. Observations:               11226   AIC:                            -2336.
Df Residuals:                   11220   BIC:                            -2292.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0505      0.002     22.087      0.0

In [ ]:
import statsmodels.api as sm

df_X = df.drop('出没回数', axis=1)
df_y = df['出没回数']

df_X = sm.add_constant(df_X)

model = sm.OLS(df_y, df_X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   出没回数   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.7430
Date:                Mon, 10 Jan 2022   Prob (F-statistic):              0.591
Time:                        06:35:39   Log-Likelihood:                -3579.1
No. Observations:                1872   AIC:                             7170.
Df Residuals:                    1866   BIC:                             7203.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3018      0.042      7.182      0.0